## Импортирование библиотек:

In [1]:
import html5lib
import requests
from bs4 import BeautifulSoup, Tag
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Константные значения и поля:

In [2]:
LINK = "https://habr.com" # источнимк взятия статей, в данном примере Habr
OBJECTS = [ # темы статей, на которых мы будем обучать модели, а также их предсказывать 
    "SQL",
    "Java",
    "Python",
]
RANDOM_STATE = 255 # для фиксирования результатов
PAGES = 100 # количество проссматриваемых страниц на хабре для каждой темы

## Дополнительные функции:

Функция поиска текстов из Habr по objects - списку поисковых тем:

In [3]:
import requests
from bs4 import BeautifulSoup

def LoadTextOfHabr(link, objects):
    data= []
    for obj in objects:
        links = []
        print("URL: \n")
        for i in range(1,PAGES+1):
            resp = requests.get(link+"/en/search/page"+str(i)+"/?q="+obj)
            print(resp.url)
            if resp.status_code != 200:
                continue
            soup = BeautifulSoup(resp.text, "html5lib")
            for a in soup.findAll('a'):
                if ((str(a).find("news/") > -1) | (str(a).find("blog/") > -1) | (str(a).find("post/") > -1)) & ~(str(a).find("comments") > -1):
                    links.append(a["href"])
        print("Links:\n")
        for l in links:
            resp = requests.get(link+l)
            print(resp.url)
            if resp.status_code != 200:
                continue
            text = BeautifulSoup(resp.text, "html5lib").get_text()
            data.append([text, obj])
    return data

## Импортирование и предобработка данных:

In [4]:
data_load = LoadTextOfHabr(LINK, OBJECTS)

URL: 

https://habr.com/en/search/page1/?q=SQL
https://habr.com/en/search/page2/?q=SQL
https://habr.com/en/search/page3/?q=SQL
https://habr.com/en/search/page4/?q=SQL
https://habr.com/en/search/page5/?q=SQL
https://habr.com/en/search/page6/?q=SQL
https://habr.com/en/search/page7/?q=SQL
https://habr.com/en/search/page8/?q=SQL
https://habr.com/en/search/page9/?q=SQL
https://habr.com/en/search/page10/?q=SQL
https://habr.com/en/search/page11/?q=SQL
https://habr.com/en/search/page12/?q=SQL
https://habr.com/en/search/page13/?q=SQL
https://habr.com/en/search/page14/?q=SQL
https://habr.com/en/search/page15/?q=SQL
https://habr.com/en/search/page16/?q=SQL
https://habr.com/en/search/page17/?q=SQL
https://habr.com/en/search/page18/?q=SQL
https://habr.com/en/search/page19/?q=SQL
https://habr.com/en/search/page20/?q=SQL
https://habr.com/en/search/page21/?q=SQL
https://habr.com/en/search/page22/?q=SQL
https://habr.com/en/search/page23/?q=SQL
https://habr.com/en/search/page24/?q=SQL
https://habr.com/e

https://habr.com/en/search/page70/?q=Java
https://habr.com/en/search/page71/?q=Java
https://habr.com/en/search/page72/?q=Java
https://habr.com/en/search/page73/?q=Java
https://habr.com/en/search/page74/?q=Java
https://habr.com/en/search/page75/?q=Java
https://habr.com/en/search/page76/?q=Java
https://habr.com/en/search/page77/?q=Java
https://habr.com/en/search/page78/?q=Java
https://habr.com/en/search/page79/?q=Java
https://habr.com/en/search/page80/?q=Java
https://habr.com/en/search/page81/?q=Java
https://habr.com/en/search/page82/?q=Java
https://habr.com/en/search/page83/?q=Java
https://habr.com/en/search/page84/?q=Java
https://habr.com/en/search/page85/?q=Java
https://habr.com/en/search/page86/?q=Java
https://habr.com/en/search/page87/?q=Java
https://habr.com/en/search/page88/?q=Java
https://habr.com/en/search/page89/?q=Java
https://habr.com/en/search/page90/?q=Java
https://habr.com/en/search/page91/?q=Java
https://habr.com/en/search/page92/?q=Java
https://habr.com/en/search/page93/

https://habr.com/en/news/505096/
https://habr.com/en/news/527858/
https://habr.com/en/news/695298/
https://habr.com/en/news/473926/
https://habr.com/en/news/581862/
https://habr.com/en/news/582682/
https://habr.com/en/companies/yandex_praktikum/news/669486/
https://habr.com/en/news/684376/
https://habr.com/en/companies/southbridge/news/703652/
https://habr.com/en/companies/southbridge/news/705358/
https://habr.com/en/companies/yandex_praktikum/news/720760/
https://habr.com/en/news/519414/
https://habr.com/en/news/645777/
https://habr.com/en/companies/southbridge/news/666476/
https://habr.com/en/companies/southbridge/news/710188/
https://habr.com/en/companies/southbridge/news/656419/
https://habr.com/en/news/662437/
https://habr.com/en/companies/southbridge/news/680662/
https://habr.com/en/companies/southbridge/news/690692/
https://habr.com/en/companies/selectel/news/724534/
https://habr.com/en/companies/netologyru/news/725226/
https://habr.com/en/companies/southbridge/news/580100/
http

In [5]:
data = pd.DataFrame(data_load, columns = ["text", "value"])

In [6]:
data[data.value == "SQL"]

,text,value
0,\n \n \n \n Ошибка в экспоненциальной форм...,SQL
1,\n \n \n \n Яндекс Практикум запускает кур...,SQL
2,"\n \n \n \n Gals Software, Москва - Систем...",SQL
3,"\n \n \n \n Python, SQL, С и Java стали са...",SQL
4,\n \n \n \n DataLine запускает кластер для...,SQL
5,\n \n \n \n Бесплатный курс-симулятор: вве...,SQL
6,\n \n \n \n SQL Camp — летняя образователь...,SQL
7,\n \n \n \n Разработчики Django исправили ...,SQL
8,\n \n \n \n Открытый Турнир по программиро...,SQL
9,\n \n \n \n Внимание! Приглашаем программи...,SQL


Как видим получилось создать выборку данных на основе текстов из сайта Habr по интерисующим темам OBJECTS и закинуть их в dataframe

## Разбиение на тестовую и обучающую выборки:

In [7]:
X = data.text
Y = data.value
Y = Y.map(lambda x: OBJECTS.index(x) if x in OBJECTS else x) # замена названий тем в виде их 
                                                             # порядкового номера в списке OBJECTS
count_vect = CountVectorizer()
X_count = count_vect.fit_transform(X)
tf_transformer = TfidfTransformer(use_idf=False).fit(X_count)
X = tf_transformer.transform(X_count)
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.30, random_state=RANDOM_STATE)

## Обучение модели:

In [8]:
model = SGDClassifier(random_state=RANDOM_STATE, max_iter=1000, n_jobs = 3)
model.fit(x_train, y_train);

## Проверка результатов:

In [9]:
pred = model.predict(x_test)
print("accuracy_score:", accuracy_score(pred, y_test))
print("classification_report:\n", classification_report(pred, y_test))

accuracy_score: 0.8378378378378378
classification_report:
               precision    recall  f1-score   support

           0       0.50      0.67      0.57         6
           1       0.93      0.88      0.90        16
           2       0.93      0.87      0.90        15

    accuracy                           0.84        37
   macro avg       0.79      0.80      0.79        37
weighted avg       0.86      0.84      0.85        37



Как видно из примера модель SGDClassifier определяет тип описанного в нем языка программирования, по содержанию статьи примерно на 60-80 процентов 